In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.request import urlopen
import pandas as pd

In [2]:
def scrape_timesjob(keywords, location):
    keywords = keywords.replace(" ","%20").lower()
    location = location.replace(" ","%20").lower()
    df = pd.DataFrame()
    sequence = 1
    job_links = []
    min_salaries = []
    max_salaries = []
    
    while True and sequence<3:
        url = f"https://www.timesjobs.com/candidate/job-search.html?from=submit&actualTxtKeywords={keywords}&searchBy=0&rdoOperator=OR&searchType=personalizedSearch&txtLocation={location}&luceneResultSize=25&postWeek=60&txtKeywords=web%20dev&pDate=I&sequence={sequence}&startPage=1"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        job_titles = [job.text.strip() for job in soup.find_all("h2")]
        if not job_titles:
            break
        job_links = [a.get("href") for h2 in soup.find_all("h2") for a in h2.find_all("a")]
        companies = [company.text.strip().split("\r\n", 1)[0] for company in soup.find_all("h3", class_="joblist-comp-name")]
        dpls = [dpl.text.strip() for dpl in soup.find_all("ul", class_="top-jd-dtl clearfix")]
        
        experiences = []
        salaries = []
        locations = []
        
        for dpl in dpls:
            # Extract years of experience
            experience_match = re.search(r'(\d+\s*-\s*\d+)\s+yrs', dpl)
            if experience_match:
                experience = experience_match.group(1) + " years"
            else:
                experience = "N/A"
            experiences.append(experience)

            # Extract salary range
            salary_match = re.search(r'₹(.+?)\n', dpl)
            if salary_match:
                salary = salary_match.group(1)
            else:
                salary = "N/A"
            salaries.append(salary)

            # Extract locations
            locations_match = re.search(r'location_on\n(.+)', dpl)
            if locations_match:
                locations_list = re.findall(r'\w[\w\s/,-]+', locations_match.group(1))
                locations.append(str(locations_list).replace("[", "").replace("]", "").replace("\'", ""))
            else:
                locations.append("N/A")
        
        min_salary = [salary.split(' - ')[0].replace('Rs', '').strip() if salary != "N/A" else "N/A" for salary in salaries]
        max_salary = [salary.split(' - ')[1].replace('Lacs p.a.', '').strip() if salary != "N/A" else "N/A" for salary in salaries]
        
        df_temp = pd.DataFrame({"Job Title": job_titles[:-1],
                                "Company": companies,
                                "Experience": experiences,
                                "Min Salary (LPA)": min_salary,
                                "Max Salary (LPA)": max_salary,
                                "Location(s)": locations,
                                "Link": job_links})
        
        df = pd.concat([df, df_temp])
        
        sequence += 1
    
    return df.reset_index(drop=True)

In [3]:
keywords = input("Enter keywords: ")
location = input("Enter location: ")

Enter keywords: Web Development
Enter location: Delhi


In [5]:
tj = scrape_timesjob(keywords, location)
if True:
    tj = tj[tj['Min Salary (LPA)'] != 'N/A']
tj

,Job Title,Company,Experience,Min Salary (LPA),Max Salary (LPA),Location(s),Link
0,Hiring Web Data Analyst Intern,mudrabiz,0 - 3 years,0.60,2.40,"Bengaluru / Bangalore, Delhi/NCR, Kolkata, ...",https://www.timesjobs.com/job-detail/hiring-we...
1,JAVA FULL STACK DEVELOPER,Electrobrain modern technologies pvt ltd,2 - 7 years,3.20,6.60,"Ahmedabad, Bengaluru / Bangalore, Chennai, ...",https://www.timesjobs.com/job-detail/java-full...
2,"Website Developer , Web Developer , Web Designer",AppSoft Solutions,0 - 3 years,6.50,10.50,"Bengaluru / Bangalore, Delhi, Mumbai, Noida...",https://www.timesjobs.com/job-detail/website-d...
3,"Team Lead , Software Engineer , Developer , Co...",ARBAJ,4 - 9 years,9.20,21.75,"Bengaluru / Bangalore, Gurgaon, Hyderabad/Se...",https://www.timesjobs.com/job-detail/team-lead...
4,Operator Jobs / In dheli -MIS Executive,Priyanka Vinayak,0 - 1 years,1.00,2.00,"Delhi, Gurgaon, Guntakal, Itanagar, Sibsagar",https://www.timesjobs.com/job-detail/operator-...
5,HomeBased / Computer Operator Jobs,Priyanka Vinayak,0 - 1 years,1.00,2.00,"Delhi, Daman , Diu, Cuddapah, Itanagar, Gu...",https://www.timesjobs.com/job-detail/homebased...
6,Online Job For Students,sp solutions,0 - 3 years,0.50,5.60,"Chennai, Delhi",https://www.timesjobs.com/job-detail/online-jo...
7,"""Java Full stack developer""",E FRIEND TECHNOLOGIES,5 - 10 years,20.00,40.00,"Delhi/NCR, Hyderabad/Secunderabad, Mumbai, ...",https://www.timesjobs.com/job-detail/java-full...
9,"Web Designing , Web Designer , Graphics Designer",AppSoft Solutions,0 - 3 years,2.40,4.50,"Ahmedabad, Chennai, Gurgaon, Kolkata, Noid...",https://www.timesjobs.com/job-detail/web-desig...
10,Engineer Associate,Laxmi Infotech,1 - 6 years,3.00,8.00,"Ahmedabad, Delhi, Gurgaon, Kolkata, Noida/...",https://www.timesjobs.com/job-detail/engineer-...
